# Executive Summary
---

# Preprocessing & Modeling
---

## Load in final data for preprocessing and modeling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt 
from matplotlib.patches import Patch
plt.style.use('fivethirtyeight')

In [3]:
final_reviews_df = pd.read_csv('../data/final_all_console_reviews.csv')

final_reviews_df.head()

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score,target
0,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,this site is a joke this the first time when i...,9,1
1,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,fair review of rdr2 im almost <number> finishe...,7,1
2,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,i really wanted to love it the overworld is be...,6,1
3,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,beautiful graphics excellent voice acting lots...,7,1
4,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,this game is really overrated the amazing envi...,7,1


## Let's take a look at some of the columns from our EDA to see if we need to make any edits.

## `avg_user_score` had potential outliers below 6.5 lets take a closer look

In [4]:
final_reviews_df.target.value_counts()

1    57740
0    54605
Name: target, dtype: int64

In [5]:
final_reviews_df[final_reviews_df['avg_user_score'] > 6.05].target.value_counts()

1    57740
0    48002
Name: target, dtype: int64

### Removing these outliers removes 5681 reviews with a target of 0. Let's go ahead and drop these from our dataframe.

In [6]:
final_reviews_df.drop((final_reviews_df[final_reviews_df['avg_user_score'] < 6.05].index), inplace=True)

## `critic_score` had some potential outliers (scores below 78), let's take a look

In [7]:
final_reviews_df.target.value_counts()

1    57740
0    48002
Name: target, dtype: int64

In [8]:
final_reviews_df[final_reviews_df['critic_score'] > 78].target.value_counts()

1    57718
0    47016
Name: target, dtype: int64

### Removing outliers in critic score removes 1826 reviews, primarily reviews with a target of 0. Let's go ahead and drop these rows from our main dataframe

In [9]:
final_reviews_df.drop((final_reviews_df[final_reviews_df['critic_score'] < 78].index), inplace=True)

## `user_score` had some potential outliers (scores below 2.5), let's take a look

In [10]:
final_reviews_df.target.value_counts()

1    57718
0    47120
Name: target, dtype: int64

In [11]:
final_reviews_df[final_reviews_df['user_score'] > 2.5].target.value_counts()

1    54506
0    39923
Name: target, dtype: int64

In [12]:
final_reviews_df[final_reviews_df['user_score'] < 2.5]['video_game_name'].value_counts()[-100:-50]

Mortal Kombat X                                                              4
Chivalry 2                                                                   4
Warframe                                                                     4
Injustice 2: Legendary Edition                                               4
Forza Horizon 2                                                              4
Astro Bot: Rescue Mission                                                    4
Bug Fables: The Everlasting Sapling                                          4
The Orange Box                                                               4
Control: Ultimate Edition                                                    4
DiRT Rally                                                                   4
The King of Fighters XV                                                      4
DmC: Devil May Cry Definitive Edition                                        4
Metro Exodus: Complete Edition                      

### Removing these outliers removes 10409 reviews. The large majority coming from reviews with a target of 0. Most of these correspond to only a few reviews per game. Let's remove these outliers.

In [13]:
final_reviews_df.drop((final_reviews_df[final_reviews_df['user_score'] < 2.5].index), inplace=True)

## Now let's dummy our categorical columns starting with `console` and `esrb_rating`

In [15]:
final_reviews_df = \
pd.get_dummies(final_reviews_df, columns=['console', 'esrb_rating'], prefix=['console', 'rating'], drop_first=True)

## Next column to dummy is `video_game_name`

In [16]:
final_reviews_df['video_game_name'].nunique()

370

### Since there are 370 unique video game names we are going to create a custom column, "appears in top 40 most frequent"

In [17]:
video_game_name_top_40_filter = final_reviews_df['video_game_name'].value_counts()[:40].index
video_game_name_top_40_filter

Index(['Red Dead Redemption 2', 'Elden Ring', 'The Witcher 3: Wild Hunt',
       'Grand Theft Auto V', 'DOOM Eternal',
       'Metal Gear Solid V: The Phantom Pain', 'Assassin's Creed Valhalla',
       'Fallout 4', 'Undertale', 'Sekiro: Shadows Die Twice',
       'Forza Horizon 5', 'God of War', 'Batman: Arkham Knight',
       'Resident Evil 2', 'Portal 2', 'Bloodborne',
       'Uncharted 4: A Thief's End', 'The Legend of Zelda: Breath of the Wild',
       'Marvel's Spider-Man', 'Super Mario Odyssey', 'Persona 5',
       'Astral Chain', 'Overwatch', 'Team Fortress 2', 'Metroid Dread',
       'Half-Life 2', 'BioShock Infinite', 'Minecraft',
       'Horizon Forbidden West', 'BioShock', 'Super Smash Bros. Ultimate',
       'Horizon Zero Dawn', 'Demon's Souls', 'Fire Emblem: Three Houses',
       'The Elder Scrolls V: Skyrim', 'Resident Evil Village',
       'Persona 5 Royal', 'Ratchet & Clank: Rift Apart', 'NieR: Automata',
       'The Last of Us Remastered'],
      dtype='object')

In [18]:
final_reviews_df['video_game_name_in_top_40'] = \
[1 if game in video_game_name_top_40_filter else 0 for game in final_reviews_df['video_game_name']]

In [19]:
final_reviews_df.drop(columns=['video_game_name'], inplace=True)

## Next column to dummy is `developer`

In [20]:
final_reviews_df['developer'].nunique()

221

### There are 221 unique developers, let's create a custom column, "appears in top 25 most frequent"

In [21]:
developer_top_25_filter = final_reviews_df['developer'].value_counts()[:25].index
developer_top_25_filter

Index(['From Software', 'Valve Software', 'Nintendo', 'BioWare',
       'Blizzard Entertainment', 'Bethesda Game Studios',
       'CD Projekt Red Studio', 'Capcom', 'Insomniac Games', 'Rockstar North',
       'Rockstar Games', 'Square Enix', 'PlatinumGames', 'Playground Games',
       'id Software', 'Naughty Dog', 'Irrational Games', 'Guerrilla', 'Atlus',
       'Respawn Entertainment', 'Rocksteady Studios', 'Kojima Productions',
       'Arkane Studios', 'JapanStudio', 'Ubisoft Montreal'],
      dtype='object')

In [22]:
final_reviews_df['developer_in_top_25'] = \
[1 if developer in developer_top_25_filter else 0 for developer in final_reviews_df['developer']]

In [23]:
final_reviews_df.drop(columns=['developer'], inplace=True)

## Finally let's look at the `genre(s)` column

In [24]:
final_reviews_df['genre(s)'].nunique()

167

### There are 167 unique genres, let's create a custom column, "appears in top 20 most frequent"

In [25]:
genre_top_20_filter = final_reviews_df['genre(s)'].value_counts()[:20].index
genre_top_20_filter

Index(['Role-Playing, Action RPG', 'Action Adventure, Open-World',
       'Action, Shooter, Shooter, First-Person, Sci-Fi, Sci-Fi, Arcade',
       'Action Adventure, General', 'Action, Shooter, First-Person, Arcade',
       'Action, Platformer, 3D', 'Action RPG, Role-Playing, Action RPG',
       'Role-Playing, Japanese-Style', 'Action, Platformer, 2D',
       'Role-Playing, First-Person, First-Person, Western-Style',
       'Action Adventure, Linear', 'Racing, Simulation, Automobile',
       'Action Adventure, Survival',
       'Role-Playing, Action Adventure, Action RPG, Open-World',
       'Action, Fighting, 2D', 'General, Role-Playing, Western-Style',
       'Action Adventure, Modern, Open-World',
       'Role-Playing, PC-style RPG, PC-style RPG, Western-Style',
       'Action, Platformer, 2D, , Metroidvania',
       'Action Adventure, Adventure, 3D, Sandbox, First-Person, Fantasy, Fantasy'],
      dtype='object')

In [26]:
final_reviews_df['genre_in_top_20'] = \
[1 if genre in genre_top_20_filter else 0 for genre in final_reviews_df['genre(s)']]

In [27]:
final_reviews_df.drop(columns=['genre(s)'], inplace=True)

In [28]:
final_reviews_df.head(2)

,summary,num_players,critic_score,avg_user_score,user_review,user_score,target,console_ps4,console_ps5,console_switch,console_xbox-series-x,console_xboxone,rating_E10+,rating_M,rating_T,video_game_name_in_top_40,developer_in_top_25,genre_in_top_20
0,developed by the creators of grand theft auto ...,32,97,8.6,this site is a joke this the first time when i...,9,1,1,0,0,0,0,0,1,0,1,1,1
1,developed by the creators of grand theft auto ...,32,97,8.6,fair review of rdr2 im almost <number> finishe...,7,1,1,0,0,0,0,0,1,0,1,1,1


In [29]:
# reviews_df[reviews_df['video_game_name'] == 'MotoGP 22'].sample(n=1000, replace=True)